In [38]:
import os
import geopandas as gpd
from blocksnet import GraphGenerator
import blocksnet
from blocksnet.preprocessing import DataGetter, AggregateParameters

from blocksnet.models import City
import pandas as pd
from shapely import Point


import warnings
warnings.filterwarnings('ignore')



local_crs = 32636
example_data_path = "./my_data"

In [39]:
blocks = gpd.read_parquet(os.path.join(example_data_path, 'blocks_cutter_result.parquet')).to_crs(local_crs)
blocks.head()

,geometry,id,landuse
0,"POLYGON ((303022.702 6675775.060, 303022.795 6...",0,no_dev_area
1,"POLYGON ((303012.608 6675801.562, 303012.586 6...",1,no_dev_area
2,"POLYGON ((303064.339 6675806.849, 303064.442 6...",2,no_dev_area
3,"POLYGON ((303428.643 6675829.284, 303425.776 6...",3,buildings
4,"POLYGON ((303429.370 6675800.664, 303429.236 6...",4,no_dev_area


In [40]:
houses = gpd.read_file(os.path.join(example_data_path, "house_points.geojson")).to_crs(local_crs)
houses.head()

,avg_m2_price_rent,mean_price_rent,P_clinic,P_hospitals,P_kindergarten,P_mfc,P_schools,accs_time,log_accs_time,Iq,P_avg,Idx,geometry
0,1190.476190,25000.0,1,1,1.0,0.0,1.0,40.8,1.61,0.55,0.8,0.675,POINT (349844.614 6646516.903)
1,537.634409,30000.0,1,1,1.0,0.0,1.0,83.0,1.92,0.37,0.8,0.585,POINT (349100.178 6657288.179)
2,708.573345,44666.0,1,1,1.0,0.0,1.0,10.0,1.00,0.50,0.8,0.650,POINT (342768.230 6637807.041)
3,538.461538,35000.0,1,1,1.0,0.0,1.0,10.0,1.00,0.61,0.8,0.705,POINT (342687.422 6637725.165)
4,2000.000000,32000.0,1,1,0.0,1.0,0.0,39.5,1.60,0.42,0.6,0.510,POINT (350044.641 6646061.322)


In [41]:
houses['Iq'].describe()

count    1505.000000
mean        0.405654
std         0.088457
min         0.300000
25%         0.340000
50%         0.380000
75%         0.490000
max         0.790000
Name: Iq, dtype: float64

In [42]:
houses = houses.loc[houses['Iq'] > 0.3]
houses['Iq'].describe()

count    1404.000000
mean        0.413255
std         0.086756
min         0.310000
25%         0.350000
50%         0.380000
75%         0.490000
max         0.790000
Name: Iq, dtype: float64

In [43]:
houses_pol = houses.copy()
houses_pol['geometry'] = houses['geometry'].apply(lambda x: x.buffer(0.001))  # Преобразование точек в небольшие полигоны

# Выполнение пространственного объединения для определения, в каких кварталах находятся дома
overlay = gpd.overlay(blocks, houses_pol, how="intersection")

unique_blocks_ids = overlay["id"].unique()

blocks_with_houses = blocks[blocks["id"].isin(unique_blocks_ids)]
blocks_with_houses.head()

,geometry,id,landuse
2854,"POLYGON ((337030.970 6638333.827, 337031.347 6...",2854,selected_area
3326,"POLYGON ((339076.063 6636586.371, 339075.615 6...",3326,no_dev_area
3331,"POLYGON ((338883.944 6636688.475, 338884.040 6...",3331,buildings
3503,"POLYGON ((339697.103 6635459.892, 339690.468 6...",3503,no_dev_area
3504,"POLYGON ((340239.631 6635418.232, 340239.373 6...",3504,no_dev_area


In [44]:
blocks_with_houses.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 241 entries, 2854 to 16014
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  241 non-null    geometry
 1   id        241 non-null    int64   
 2   landuse   241 non-null    object  
dtypes: geometry(1), int64(1), object(1)
memory usage: 7.5+ KB


In [45]:
# blocks_with_houses.explore()

In [46]:
getter = DataGetter(blocks=blocks_with_houses)

In [47]:
# Создание графа транспортной доступности 
# +
# Создание матрицы доступности и сохранение ее в пикл
'''
city_geometry = gpd.GeoDataFrame([{'geometry': blocks_with_houses.unary_union.convex_hull}]).set_crs(local_crs)
intermodal_graph = GraphGenerator(city_geometry=city_geometry, local_crs=local_crs).get_graph('intermodal')
accessibility_matrix = getter.get_accessibility_matrix(intermodal_graph)
accessibility_matrix.to_pickle(os.path.join(example_data_path, "data_getter_matrix.pickle"))
'''

accessibility_matrix = pd.read_pickle(os.path.join(example_data_path, "data_getter_matrix.pickle"))
accessibility_matrix.head()

id,2854,3326,3331,3503,3504,3506,3509,3511,3512,3513,...,15348,15366,15375,15383,15560,15613,15626,15773,16003,16014
id,,,,,,,,,,,,,,,,,,,,,
2854,0.0,28.5,29.3,33.0,35.6,33.3,27.7,28.7,28.6,33.2,...,98.2,111.3,107.2,111.9,97.5,105.3,111.1,103.5,114.3,115.4
3326,26.5,0.0,11.4,18.4,24.5,21.0,12.1,11.3,8.4,18.8,...,90.2,103.3,97.5,103.3,89.5,97.3,103.1,95.5,104.7,107.3
3331,27.6,11.4,0.0,24.4,27.0,24.7,19.1,20.3,19.8,24.6,...,92.7,105.8,100.0,105.8,92.0,99.8,105.6,98.0,107.2,109.8
3503,30.6,18.4,23.8,0.0,11.3,4.9,13.9,16.7,17.9,21.0,...,89.0,102.1,96.3,102.1,88.3,96.1,101.9,94.3,103.5,106.1
3504,31.3,21.6,24.5,11.3,0.0,7.1,14.9,17.4,19.8,20.7,...,86.2,99.3,93.5,99.3,85.5,93.3,99.1,91.5,100.7,103.3


In [48]:
# city_geometry = gpd.GeoDataFrame([{'geometry': blocks_with_houses.unary_union.convex_hull}]).set_crs(local_crs)
# intermodal_graph = GraphGenerator(city_geometry=city_geometry, local_crs=local_crs).get_graph('intermodal')
# gg = blocksnet.preprocessing.graph_generator.GraphGenerator(city_geometry = city_geometry)
# gg.plot(intermodal_graph)

In [63]:
buildings = gpd.read_parquet(os.path.join(example_data_path, "all_buildings_spb.parquet")).to_crs(local_crs)
buildings.head()

,building_id,building_area,living_area,storeys_count,population_balanced,is_living,geom
0,1,145.871,NaN,2.0,29,False,"MULTIPOLYGON (((302685.489 6676297.801, 302689..."
1,2,139.242,NaN,NaN,0,False,"POLYGON ((302929.515 6676138.437, 302932.880 6..."
2,3,390.244,NaN,NaN,0,True,"MULTIPOLYGON (((303042.071 6676392.771, 303043..."
3,4,579.998,405.998,1.0,0,True,"MULTIPOLYGON (((303078.321 6676453.630, 303091..."
4,5,695.979,915.400,1.0,151,False,"MULTIPOLYGON (((303246.716 6676494.615, 303278..."


In [64]:
# Выполнение пространственного объединения для определения нужных домов
overlay = gpd.overlay(buildings, houses_pol, how="intersection")

unique_buildings_ids = overlay["building_id"].unique()

buildings = buildings[buildings["building_id"].isin(unique_buildings_ids)]
buildings.head()

,building_id,building_area,living_area,storeys_count,population_balanced,is_living,geom
24277,24510,1332.330,6548.3,9.0,188,True,"MULTIPOLYGON (((338091.065 6638065.940, 338095..."
24886,25123,381.492,692.5,3.0,20,True,"MULTIPOLYGON (((338565.066 6637860.958, 338567..."
25425,25664,339.437,3735.0,15.0,107,True,"MULTIPOLYGON (((339056.394 6637144.906, 339065..."
25525,25764,439.492,4833.9,15.0,139,True,"MULTIPOLYGON (((339132.623 6637136.638, 339139..."
25550,25789,804.020,3525.1,5.0,101,True,"MULTIPOLYGON (((339135.481 6636807.653, 339136..."


In [70]:
buildings.shape

(1397, 7)

In [72]:
houses.shape

(1404, 13)

In [69]:
greenings = gpd.GeoDataFrame([{'current_green_area': 0, 'current_green_capacity': 0, 'geometry': Point(0,0)}]).set_crs(local_crs)
parkings = gpd.GeoDataFrame([{'current_parking_capacity': 0, 'geometry': Point(0,0)}]).set_crs(local_crs)
greenings.head()

,current_green_area,current_green_capacity,geometry
0,0,0,POINT (0.000 0.000)


In [73]:
aggr_params = AggregateParameters(
  buildings=buildings,
  greenings=greenings,
  parkings=parkings
)

aggregated_blocks = getter.aggregate_blocks_info(params=aggr_params)
aggregated_blocks.head()

GeoDataFrame CRS set to estimated: EPSG:32635
GeoDataFrame CRS set to estimated: EPSG:32635


ValidationError: 3 validation errors for AggregateParameters
buildings.geometry
  Field required [type=missing, input_value={'index': 24277, 'buildin...0.793, 338140.045 6...>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.0.1/v/missing
buildings.living_area_pyatno
  Field required [type=missing, input_value={'index': 24277, 'buildin...0.793, 338140.045 6...>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.0.1/v/missing
buildings.total_area
  Field required [type=missing, input_value={'index': 24277, 'buildin...0.793, 338140.045 6...>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.0.1/v/missing